In [1]:
from itertools import product

In [15]:
import sys
import os
sys.path.append("../") # go to parent dir
from src.Derivation import Derivation
derivator = Derivation(use_guesser=True)

Error: rule772([verb + ITFX] + noun + 0(ый) -> adj)


In [16]:
for i, rule in enumerate(derivator.rules_compound):
    print(i, rule)

0 ruleTODO([noun + ITFX] + verb + 0m2 -> noun)	(-)
1 ruleTODO([noun + ITFX] + verb + 0m2 -> noun)	(-)
2 rule550([noun + ITFX] + noun -> noun)	(-)
3 rule550([adj + ITFX] + noun -> noun)	(-)
4 rule550([num + ITFX] + noun -> noun)	(-)
5 rule552([пол] + noun + GEN -> noun)	(-)
6 rule558([noun + ITFX] + verb + тель -> noun)	(-)
7 rule558([adj + ITFX] + verb + тель -> noun)	(-)
8 rule559([noun + ITFX] + verb + ец/нец/енец/омец -> noun)	(-)
9 rule559([adj + ITFX] + verb + ец/нец/енец/омец -> noun)	(-)
10 rule559([num + ITFX] + verb + ец/нец/енец/омец -> noun)	(-)
11 rule560([noun + ITFX] + verb + щик -> noun)	(-)
12 rule561([noun + ITFX] + verb + льщик -> noun)	(-)
13 rule562([noun + ITFX] + verb + ник/еник/енник/ик -> noun)	(-)
14 rule761([noun + ITFX] + noun + н1(ый) -> adj)	(-)
15 rule761([adj + ITFX] + noun + н1(ый) -> adj)	(-)
16 rule761([num + ITFX] + noun + н1(ый) -> adj)	(-)
17 rule762([adj + ITFX] + noun + н(ий) -> adj)	(-)
18 rule762([num + ITFX] + noun + н(ий) -> adj)	(-)
19 rule76

In [17]:
rule = derivator.rules_compound[8]
print(rule)

rule559([noun + ITFX] + verb + ец/нец/енец/омец -> noun)	(-)


In [18]:
def get_words(rule, word_b, pos_b, words_m, poss_m, pos_a, **kwargs):    
    tags = [derivator.tag_guesser.guess(word=word_m, pos=pos_m) or [{}]
            for (word_m, pos_m) in zip([word_b] + words_m, [pos_b] + poss_m)]
    results = set()
    for tag_comb in product(*tags):
        results |= rule.apply_with_tags(word_b=word_b, pos_b=pos_b, pos_a=pos_a, words_m=words_m, poss_m=poss_m, tags_dict=tag_comb, **kwargs)
    return results

In [22]:
get_words(rule, 'любить', 'verb', ['победа'], ['noun'], 'noun', use_rare=True)

{'побдолюбец', 'победолюбец'}